In [85]:
import pandas as pd
import numpy as np
import warnings

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.feature_selection import RFE

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [86]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/svm_classifier.csv')

In [87]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [88]:
### Season to test results

N = 2021

In [89]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [90]:
def score_classification(model):
    predictions = []
    prob = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        predictions += list(prediction_df['predicted'].values)  
        prob += list(prediction_df['proba_1'].values)

    return predictions, prob

In [91]:
parameters.head()

,model,gamma,C,kernel,score
0,svm_classifier,0.000886,1.128838,sigmoid,0.684211
1,svm_classifier,0.000886,0.127427,rbf,0.684211
2,svm_classifier,0.000144,0.784760,linear,0.631579
3,svm_classifier,0.000298,0.784760,linear,0.631579
4,svm_classifier,0.000207,0.784760,linear,0.631579


In [92]:
params = parameters.iloc[0]

gamma = params.gamma
c = params.C
kernel = params.kernel

params

model     svm_classifier
gamma           0.000886
C               1.128838
kernel           sigmoid
score           0.684211
Name: 0, dtype: object

In [93]:
model = SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions, probs = score_classification(model)

In [94]:
df = processed.copy()
df = df[(df.season == N)]

df['predicted'] = predictions
df['proba_1'] = probs

In [95]:
df.query('predicted == 1')

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,driver_points_before,constructor_points_before,driver_points_from,constructor_points_from,driver_points_per,points_percentage,driver_last_3,constructor_last_3,predicted,proba_1
2743,2021,1,bahrain,max_verstappen,red_bull,2,18,0,2,28,...,0.0,0.0,18.0,28.0,0.00,0.00,0.0,0.0,1,0.241114
2763,2021,2,imola,lewis_hamilton,mercedes,2,44,1,1,60,...,25.0,41.0,19.0,19.0,12.50,60.98,25.0,41.0,1,0.393281
2781,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,44.0,60.0,25.0,41.0,14.67,73.33,44.0,60.0,1,0.360308
2801,2021,4,catalunya,lewis_hamilton,mercedes,1,94,3,1,141,...,69.0,101.0,25.0,40.0,17.25,68.32,69.0,101.0,1,0.442254
2821,2021,5,monaco,max_verstappen,red_bull,1,105,2,1,149,...,80.0,112.0,25.0,37.0,16.00,71.43,62.0,84.0,1,0.354253
2853,2021,6,baku,lewis_hamilton,mercedes,15,101,3,2,148,...,101.0,148.0,0.0,0.0,16.83,68.24,57.0,88.0,1,0.328188
2858,2021,7,ricard,max_verstappen,red_bull,1,131,3,1,215,...,105.0,174.0,26.0,41.0,15.00,60.34,44.0,91.0,1,0.307644
2876,2021,8,red_bull_ring,max_verstappen,red_bull,1,156,4,1,252,...,131.0,215.0,25.0,37.0,16.38,60.93,51.0,103.0,1,0.309703
2896,2021,9,red_bull_ring,max_verstappen,red_bull,1,182,5,1,286,...,156.0,252.0,26.0,34.0,17.33,61.90,51.0,103.0,1,0.307371
2917,2021,11,hungaroring,lewis_hamilton,mercedes,2,195,4,1,303,...,177.0,285.0,18.0,18.0,16.09,62.11,58.0,107.0,1,0.290774


In [96]:
df.query('season == 2021 & round == 3').sort_values('proba_1', ascending=False)

,season,round,circuit_id,driver,constructor,podium,driver_points_after,driver_wins_after,driver_standings_pos_after,constructor_points_after,...,driver_points_before,constructor_points_before,driver_points_from,constructor_points_from,driver_points_per,points_percentage,driver_last_3,constructor_last_3,predicted,proba_1
2781,2021,3,portimao,lewis_hamilton,mercedes,1,69,2,1,101,...,44.0,60.0,25.0,41.0,14.67,73.33,44.0,60.0,1,0.360308
2782,2021,3,portimao,max_verstappen,red_bull,2,61,1,2,83,...,43.0,53.0,18.0,30.0,14.33,81.13,43.0,53.0,0,0.297016
2783,2021,3,portimao,valtteri_bottas,mercedes,3,32,0,4,101,...,16.0,60.0,16.0,41.0,5.33,26.67,16.0,60.0,0,0.233925
2784,2021,3,portimao,sergio_perez,red_bull,4,22,0,6,83,...,10.0,53.0,12.0,30.0,3.33,18.87,10.0,53.0,0,0.104144
2791,2021,3,portimao,carlos_sainz,ferrari,11,14,0,8,42,...,14.0,34.0,0.0,8.0,4.67,41.18,14.0,34.0,0,0.102597
2785,2021,3,portimao,lando_norris,mclaren,5,37,0,3,53,...,27.0,41.0,10.0,12.0,9.00,65.85,27.0,41.0,0,0.084503
2787,2021,3,portimao,esteban_ocon,alpine,7,8,0,9,13,...,2.0,3.0,6.0,10.0,0.67,66.67,2.0,3.0,0,0.067856
2786,2021,3,portimao,charles_leclerc,ferrari,6,28,0,5,42,...,20.0,34.0,8.0,8.0,6.67,58.82,20.0,34.0,0,0.058374
2789,2021,3,portimao,daniel_ricciardo,mclaren,9,16,0,7,53,...,14.0,41.0,2.0,12.0,4.67,34.15,14.0,41.0,0,0.051005
2794,2021,3,portimao,lance_stroll,aston_martin,14,5,0,11,5,...,5.0,5.0,0.0,0.0,1.67,100.00,5.0,5.0,0,0.035000
